In [11]:
# ============================================================
# CS-340 Animal Shelter Dashboard + Analytics Enhancements
# Author: Steve Arevalo
# ============================================================

# Imports
from jupyter_dash import JupyterDash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px
import dash_leaflet as dl
import base64
import matplotlib.pyplot as plt

# Import enhanced AnimalShelter class
from AnimalShelter import AnimalShelter

# MongoDB Credentials
username = "aacuser"
password = "Frontporch1"

# Instantiate class for database interaction
shelter = AnimalShelter(username, password)

# Rescue Filters
rescue_filters = {
    'water': {
        "breed": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"],
        "sex": "Intact Female",
        "age_min": 26,
        "age_max": 156
    },
    'mountain': {
        "breed": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog",
                  "Siberian Husky", "Rottweiler"],
        "sex": "Intact Male",
        "age_min": 26,
        "age_max": 156
    },
    'disaster': {
        "breed": ["Doberman Pinscher", "German Shepherd", "Golden Retriever",
                  "Bloodhound", "Rottweiler"],
        "sex": "Intact Male",
        "age_min": 20,
        "age_max": 300
    }
}

# Encode Logo
image_filename = 'SalvareLogo.png'
encoded_logo = base64.b64encode(open(image_filename, 'rb').read()).decode()

# ============================================================
# DASHBOARD SETUP
# ============================================================

app = JupyterDash(__name__)

app.layout = html.Div([
    # Header section
    html.Div([
        html.A(
            html.Img(src='data:image/png;base64,{}'.format(encoded_logo),
                     style={'height': '80px'}),
            href='https://www.snhu.edu',
            target='_blank',
            title='Grazioso Salvare Home'
        ),
        html.Div("Created by Steve Arevalo", style={
            'fontSize': '18px', 'fontWeight': 'bold',
            'paddingLeft': '20px', 'alignSelf': 'center'
        })
    ], style={'display': 'flex', 'alignItems': 'center', 'padding': '10px 0'}),

    html.H1("Austin Animal Center Outcomes Dashboard"),

    # Dropdown filter
    html.Div([
        html.Label("Select Rescue Type:"),
        dcc.Dropdown(
            id='rescue-filter',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
                {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset (Show All Animals)', 'value': 'reset'}
            ],
            value=None,
            clearable=True,
            style={'width': '50%'}
        )
    ], style={'padding': '20px 0'}),

    # Animal Table
    dash_table.DataTable(
        id='animal-table',
        columns=[],
        data=[],
        page_size=10,
        filter_action='native',
        sort_action='native',
        row_selectable='single',
        style_table={'overflowX': 'auto'}
    ),

    html.Hr(),

    # Pie Chart + Map Side-by-Side
    html.Div(style={'display': 'flex', 'justifyContent': 'space-between'}, children=[
        html.Div(style={'width': '50%'}, children=[
            dcc.Graph(id='breed-pie-chart')
        ]),
        html.Div(style={'width': '48%'}, children=[
            html.Div(id='map-container')
        ])
    ])
])

# ============================================================
# CALLBACK: TABLE + PIE CHART
# ============================================================

@app.callback(
    [Output('animal-table', 'data'),
     Output('animal-table', 'columns'),
     Output('breed-pie-chart', 'figure')],
    [Input('rescue-filter', 'value')]
)
def update_data_table_and_pie(selected_filter):

    if selected_filter is None or selected_filter == 'reset':
        query = {}
    else:
        f = rescue_filters[selected_filter]
        query = {
            "breed": {"$in": f["breed"]},
            "sex_upon_outcome": f["sex"],
            "age_upon_outcome_in_weeks": {"$gte": f["age_min"], "$lte": f["age_max"]},
            "animal_type": "Dog"
        }

    records = shelter.read(query)

    for r in records:
        r.pop('_id', None)

    if records:
        columns = [{"name": k, "id": k} for k in records[0].keys()]
        df = pd.DataFrame(records)
    else:
        columns = []
        df = pd.DataFrame()

    if not df.empty and 'breed' in df.columns:
        fig = px.pie(df, names='breed', title='Breed Distribution')
    else:
        fig = px.pie(title="No Data Available")

    return records, columns, fig

# ============================================================
# CALLBACK: MAP
# ============================================================

@app.callback(
    Output('map-container', 'children'),
    [Input('animal-table', 'derived_virtual_data'),
     Input('animal-table', 'derived_virtual_selected_rows')]
)
def update_map(rows, selected_rows):

    if not rows:
        return html.P("No data available for map.")

    dff = pd.DataFrame(rows)
    row_idx = 0 if not selected_rows else selected_rows[0]

    if 'location_lat' not in dff or 'location_long' not in dff:
        return html.P("No location data available.")

    lat = dff.iloc[row_idx]['location_lat']
    lon = dff.iloc[row_idx]['location_long']

    if pd.isna(lat) or pd.isna(lon):
        return html.P("Selected animal has no valid location data.")

    return dl.Map(
        style={'width': '100%', 'height': '500px'},
        center=[lat, lon],
        zoom=12,
        children=[
            dl.TileLayer(),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(dff.iloc[row_idx].get('breed', 'Unknown Breed')),
                dl.Popup([
                    html.H4("Animal Details"),
                    html.P(f"Name: {dff.iloc[row_idx].get('name', 'N/A')}"),
                    html.P(f"Breed: {dff.iloc[row_idx].get('breed', 'N/A')}"),
                    html.P(f"Age (weeks): {dff.iloc[row_idx].get('age_upon_outcome_in_weeks', 'N/A')}"),
                    html.P(f"Outcome: {dff.iloc[row_idx].get('outcome_type', 'N/A')}")
                ])
            ])
        ]
    )

# ============================================================
# RUN DASHBOARD
# ============================================================

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True)

# ============================================================
# ANALYTICS ENHANCEMENT SECTION
# ============================================================

print("=======================================")
print(" ANALYTICS ENHANCEMENTS - CS340 Project")
print("=======================================")

# ---- Adoption Counts by Breed ----
print("\nTop Adopted Breeds:")
adoption_stats = shelter.analyze_adoptions()
display(adoption_stats)

# ---- Seasonal Adoption Trends ----
print("\nSeasonal Adoption Trends (Month-by-Month):")
seasonal_stats = shelter.seasonal_adoption_trends(show_plot=True)
display(seasonal_stats)
